In [18]:
gdpr_article_5_text = """
                      1. Personal data shall be:
                      a) processed lawfully, fairly and in a transparent manner in relation to the data subject (‘lawfulness, fairness and transparency’);
                      b) collected for specified, explicit and legitimate purposes and not further processed in a manner that is incompatible with those purposes; further processing for archiving purposes in the public interest, scientific or historical research purposes or statistical purposes shall, in accordance with Article 89(1), not be considered to be incompatible with the initial purposes (‘purpose limitation’);
                      c) adequate, relevant and limited to what is necessary in relation to the purposes for which they are processed (‘data minimisation’);
                      d) accurate and, where necessary, kept up to date; every reasonable step must be taken to ensure that personal data that are inaccurate, having regard to the purposes for which they are processed, are erased or rectified without delay (‘accuracy’);
                      e) kept in a form which permits identification of data subjects for no longer than is necessary for the purposes for which the personal data are processed; personal data may be stored for longer periods insofar as the personal data will be processed solely for archiving purposes in the public interest, scientific or historical research purposes or statistical purposes in accordance with Article 89(1) subject to implementation of the appropriate technical and organisational measures required by this Regulation in order to safeguard the rights and freedoms of the data subject (‘storage limitation’);
                      f) processed in a manner that ensures appropriate security of the personal data, including protection against unauthorised or unlawful processing and against accidental loss, destruction or damage, using appropriate technical or organisational measures (‘integrity and confidentiality’).
                      2. The controller shall be responsible for, and be able to demonstrate compliance with, paragraph 1 (‘accountability’).
                      """

In [19]:
!unzip /content/acl-coling-2014-corpus.zip -d /content/acl_corpus

In [20]:
import xml.etree.ElementTree as ET
import spacy
import os


nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    # Lowercase and replace newline characters
    text = text.lower().replace('\n', ' ')
    text = ' '.join(text.split())

    # Use spaCy for further processing
    doc = nlp(text)
    clean_text = ' '.join([token.lemma_ for token in doc if not token.is_punct])

    return clean_text

preprocessed_policies = []
directory = './acl_corpus/corpus/'

for filename in os.listdir(directory):
    if filename.endswith('.xml'):
        path = os.path.join(directory, filename)
        tree = ET.parse(path)
        root = tree.getroot()

        # Extracting text from each SECTION/SUBTEXT
        for section in root.findall('.//SECTION/SUBTEXT'):
            if section.text:
                processed_text = preprocess_text(section.text)
                preprocessed_policies.append(processed_text)


In [21]:
print(len(preprocessed_policies))

10501


In [22]:
!pip install -U sentence-transformers

In [23]:
!unzip /content/saved_model.zip -d /content/saved_model

Archive:  /content/saved_model.zip
   creating: /content/saved_model/saved_model/
  inflating: /content/saved_model/saved_model/vocab.txt  
  inflating: /content/saved_model/saved_model/tokenizer_config.json  
  inflating: /content/saved_model/saved_model/sentence_bert_config.json  
  inflating: /content/saved_model/saved_model/tokenizer.json  
   creating: /content/saved_model/saved_model/2_Normalize/
  inflating: /content/saved_model/saved_model/config.json  
  inflating: /content/saved_model/saved_model/modules.json  
  inflating: /content/saved_model/saved_model/config_sentence_transformers.json  
   creating: /content/saved_model/saved_model/1_Pooling/
  inflating: /content/saved_model/saved_model/1_Pooling/config.json  
  inflating: /content/saved_model/saved_model/README.md  
  inflating: /content/saved_model/saved_model/model.safetensors  
  inflating: /content/saved_model/saved_model/special_tokens_map.json  


In [26]:
from sentence_transformers import SentenceTransformer

sbert_model = SentenceTransformer('./saved_model/saved_model')


In [27]:
policy_embeddings = sbert_model.encode(preprocessed_policies)
gdpr_embeddings = sbert_model.encode([gdpr_article_5_text])


In [28]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Calculate semantic similarity (higher values mean more similarity)
similarities = cosine_similarity(policy_embeddings, gdpr_embeddings)
similar_policies_indices = np.where(similarities > 0.9)[0]

print(f"Number of policies potentially aligning with GDPR Article 5: {len(similar_policies_indices)}")


Number of policies potentially aligning with GDPR Article 5: 0


In [29]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Calculate semantic similarity (higher values mean more similarity)
similarities = cosine_similarity(policy_embeddings, gdpr_embeddings)
similar_policies_indices = np.where(similarities > 0.6)[0]

print(f"Number of policies potentially aligning with GDPR Article 5: {len(similar_policies_indices)}")


Number of policies potentially aligning with GDPR Article 5: 3


In [30]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Calculate semantic similarity (higher values mean more similarity)
similarities = cosine_similarity(policy_embeddings, gdpr_embeddings)
similar_policies_indices = np.where(similarities > 0.5)[0]

print(f"Number of policies potentially aligning with GDPR Article 5: {len(similar_policies_indices)}")


Number of policies potentially aligning with GDPR Article 5: 233


- **Embedding Models:** Other than *SBERT*, I can try other models like *GPT-3* for context-aware embeddings, *FastText* for handling out-of-vocabulary words, or domain-specific models that might be trained on legal or policy-related corpora.

- **Similarity Measures:** Beyond *cosine similarity*, explore other measures like *Euclidean distance* or *Manhattan distance* for comparing embeddings.

- **Clustering Techniques:** Apply unsupervised clustering (e.g., *K-means, DBSCAN*) to group policies based on their embeddings and analyze clusters for GDPR compliance themes.

- **Dimensionality Reduction:** Use techniques like *PCA* or *t-SNE* to reduce the dimensionality of your embeddings before applying similarity measures or clustering, which might reveal different patterns.

- **Threshold Tuning:** Experiment with different *thresholds* for considering a policy potentially compliant based on similarity scores.

- **Text Preprocessing Variations:** Test the impact of different *preprocessing steps*, such as including/excluding stopwords, using stemming versus lemmatization, or experimenting with n-grams.

- **Segmentation Strategies:** Instead of analyzing entire documents, try segmenting policies into smaller *sections or paragraphs* and assess their individual compliance.